## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["CHATBOT_PROJECT"] = os.getenv("CHATBOT_PROJECT")

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.1-8b-instant")
model

c:\Users\gaurv\Downloads\STUDY\Generative AI\LangChain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001F6798E4D70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F6798E5A90>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hey! My name is Gaurvit and I work as a Data Analyst")])

AIMessage(content='Nice to meet you, Gaurvit! As a Data Analyst, you must be dealing with a lot of interesting data and insights every day. What kind of projects do you typically work on, and what tools do you use to analyze and visualize data?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 50, 'total_tokens': 102, 'completion_time': 0.097616604, 'completion_tokens_details': None, 'prompt_time': 0.003238485, 'prompt_tokens_details': None, 'queue_time': 0.054746245, 'total_time': 0.100855089}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd1fc-c917-7c93-a06d-8255fcb7a70a-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 50, 'output_tokens': 52, 'total_tokens': 102})

In [4]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hey! My name is Gaurvit and I work as a Data Analyst"),
    AIMessage(content="Nice to meet you, Gaurvit. It's great to hear that you are a Data Analyst. What kind of data analysis do you typically work with - is it more focused on business intelligence, predictive modeling, or something else?"),
    HumanMessage(content="What is my name and what do i do?")
])

AIMessage(content='Your name is Gaurvit, and you work as a Data Analyst.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 117, 'total_tokens': 133, 'completion_time': 0.013639859, 'completion_tokens_details': None, 'prompt_time': 0.022707635, 'prompt_tokens_details': None, 'queue_time': 0.084607265, 'total_time': 0.036347494}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd1fc-ca95-7a31-8fad-901caf852392-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 117, 'output_tokens': 16, 'total_tokens': 133})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [6]:
config = {'configurable' : {'session_id':'chat1'}}

In [7]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey! My name is Gaurvit and I work as a Data Analyst")],
    config=config
)

In [8]:
response.content

"Nice to meet you, Gaurvit. Data Analyst is a fascinating field, and I'm sure you must be working with a wide range of data sets and tools to extract insights and drive business decisions. What specific areas of data analysis do you focus on, or are you interested in exploring?"

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="Who am I?")],
    config=config
)

In [10]:
response.content

'You are Gaurvit, a Data Analyst.'

In [11]:
# Changing the config and checking if it still remembers name and profession
config = {"configurable" : {"session_id":"chat2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Who am I?")],
    config=config
)

response.content

# Conclusion -> Message history doesn't stay when session is switched
# Chatbot only remembers chat history of that particular session

"Unfortunately, I don't have any information about you. This conversation has just started, and I don't know anything about your identity, personal details, or any context that might help me guess who you are.\n\nIf you'd like to share some information about yourself, I'd be happy to try and help you figure out who you are. Or, if you're thinking of a specific character, person, or entity, feel free to give me a hint, and I'll do my best to help you identify who or what you are thinking of."

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant, answer all the questions to the best of your capability in"),
    MessagesPlaceholder(variable_name="messages")
])

chain = prompt|model

In [13]:
chain.invoke({"messages":[HumanMessage(content="Hi, My name is Gaurvit")]})

AIMessage(content='Nice to meet you, Gaurvit. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77, 'completion_time': 0.036831018, 'completion_tokens_details': None, 'prompt_time': 0.004821934, 'prompt_tokens_details': None, 'queue_time': 0.054690776, 'total_time': 0.041652952}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd1fc-cef1-7080-ab96-a2bd61b363fd-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77})

In [14]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x000001F655BC9A80>, history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [15]:
config = {"configurable" : {"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi, I am Gaurvit and I am a Machine Learning Engineer")],
    config=config
)
response.content

"Nice to meet you, Gaurvit! It's great to hear that you're a Machine Learning Engineer. What brings you here today? Are you working on a specific project, looking for advice, or perhaps seeking to learn about new techniques in the field? I'm all ears and ready to help!"

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant, answer all the questions to the best of your capability in {language}"),
    MessagesPlaceholder(variable_name="messages")
])

chain = prompt|model

In [17]:
response_spanish = chain.invoke({"messages" : [HumanMessage(content="Hey, My name is Gaurvit and I am a Machine Learning Engineer")], "language" : "Spanish"})
response_spanish.content

'Hola Gaurvit, es un placer conocerte. Eres ingeniero de aprendizaje automático, ¿verdad? ¿En qué áreas de aprendizaje automático te especializas? ¿Estás trabajando en algún proyecto interesante actualmente?'

In [18]:
response = chain.invoke({"messages" : [HumanMessage(content="Hey, My name is Gaurvit and I am a Machine Learning Engineer")], "language" : "English"})
response.content

"Nice to meet you, Gaurvit. As a Machine Learning Engineer, I'm sure you're working with exciting projects and technologies. What brings you here today? Do you have any questions about Machine Learning, a specific project you're working on, or perhaps something else? I'm here to help."

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [19]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

In [20]:
config_new = {"configurable" : {"session_id" : "chat4"}}

response = with_message_history.invoke(
    {"messages" : [HumanMessage(content="Hi, My name is Gaurvit and I am a Machine Learning Engineer")], "language" : "Spanish"},
    config = config_new
)

response.content

'Hola Gaurvit, un placer conocerte. ¿En qué puedo ayudarte hoy como ingeniero de Machine Learning?'

In [21]:
response = with_message_history.invoke(
    {"messages" : [HumanMessage(content="Who am I")], "language" : "Hindi"},
    config = config_new
)

response.content

'आप कौन हैं? आपका नाम गौरवित है, और आप मशीन लर्निंग इंजीनियर हैं।'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [44]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=50,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [45]:
messages = [
    SystemMessage(content="You are an expert assistant, answer the questions to the best of your ability"),
    HumanMessage(content="Hi! I am Gaurvit"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like Tiramisu"),
    AIMessage(content="Nice"),
    HumanMessage(content="What is 5 * 3"),
    AIMessage(content="5 * 3 = 15"),
    HumanMessage(content="Thanks, the answer is correct!"),
    AIMessage(content="Welcome!")
]

trimmer.invoke(messages)

[SystemMessage(content='You are an expert assistant, answer the questions to the best of your ability', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 5 * 3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='5 * 3 = 15', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='Thanks, the answer is correct!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Welcome!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

In [46]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)

In [47]:
chain.invoke(
    {
    "messages" : messages + [HumanMessage(content="What do i like?")],
    "language" : "english"
    }
)

AIMessage(content="Unfortunately, I'm a large language model, I don't have any prior knowledge about your personal preferences or likes. I'm a new conversation each time, and I don't retain any information from previous conversations.\n\nIf you'd like to share what you like, I'd be happy to chat with you about it!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 93, 'total_tokens': 157, 'completion_time': 0.093441033, 'completion_tokens_details': None, 'prompt_time': 0.006103187, 'prompt_tokens_details': None, 'queue_time': 0.052495593, 'total_time': 0.09954422}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd207-ae9f-7061-8b8e-fcc55d8d754d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 93, 'output_tokens': 64, 'total_tokens': 157})

In [48]:
chain.invoke(
    {
    "messages" : messages + [HumanMessage(content="What math question did i ask?")],
    "language" : "english"
    }
)

AIMessage(content="You didn't ask a math question yet. This is the beginning of our conversation, and your first question was a general thank you. If you'd like to ask a math question or any other type of question, I'm here to help!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 95, 'total_tokens': 145, 'completion_time': 0.071184493, 'completion_tokens_details': None, 'prompt_time': 0.005509563, 'prompt_tokens_details': None, 'queue_time': 0.054732018, 'total_time': 0.076694056}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd207-afec-7c90-a389-aa388df265ef-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 95, 'output_tokens': 50, 'total_tokens': 145})